# Imports and Setups

In [ ]:
! pip install transformers==4.10.1

     |████████████████████████████████| 2.8 MB 3.8 MB/s 
     |████████████████████████████████| 596 kB 47.9 MB/s 
     |████████████████████████████████| 3.3 MB 41.9 MB/s 
     |████████████████████████████████| 61 kB 326 kB/s 
     |████████████████████████████████| 895 kB 53.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
! pip install nlpaug==1.1.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 405 kB 5.2 MB/s 


In [ ]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 7.5 MB/s 


In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 4.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=e3ea2c51231850a7fce23d3d599424b64a99d0ae4e4dbf4a9152909babcb4dbb
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [ ]:
! pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

  Cloning https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to /tmp/pip-req-build-yz06yxza
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git /tmp/pip-req-build-yz06yxza
     |████████████████████████████████| 50 kB 4.6 MB/s 
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8695 sha256=b4125a09ef31204a412bd2c5734e9a3b557305e0a468986b917cdf089c5a990a
  Stored in directory: /tmp/pip-ephem-wheel-cache-jrmg8tk8/wheels/ff/27/79/a53e92f32da44c8c4f51094ca156e6ff91103b7e48a341d62b
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149870 sha256=8f22a6020e26f15bb5211c25f18a1f6056b111e3d487a1cee003c7acb46420a3
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built parrot python-Levenshtein


Lendo o Arquivo

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/semeval_ironic/SemEval2018-T3-train-taskA_emoji.txt',sep = '\t')
dataset.head()

,Tweet index,Label,Tweet text
0,1,1,Sweet United Nations video. Just in time for C...
1,2,1,@mrdahl87 We are rumored to have talked to Erv...
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...
3,4,0,3 episodes left I'm dying over here
4,5,1,I can't breathe! was chosen as the most notabl...


In [ ]:
num_gen = 5
perc = 100

In [ ]:
X = dataset['Tweet text']
y = dataset['Label']

In [ ]:
base_cru_1p = dataset.copy()

Gerando Base Percentual

In [ ]:
from sklearn.model_selection import train_test_split
X_resto, X_treino_1p, y_resto, y_treino_1p = train_test_split(X, y, stratify=y, test_size=(perc/100), random_state=42) # Selecionando 1%

In [ ]:
import pandas as pd

data = {'texto': X_treino_1p, 'label': y_treino_1p}
base_cru_1p = pd.DataFrame.from_dict(data)

In [ ]:
import pandas as pd

data = {'texto': X, 'label': y}
base_cru_1p = pd.DataFrame.from_dict(data)

In [ ]:
base_cru_1p.head()

,texto,label
0,Sweet United Nations video. Just in time for C...,1
1,@mrdahl87 We are rumored to have talked to Erv...,1
2,Hey there! Nice to see you Minnesota/ND Winter...,1
3,3 episodes left I'm dying over here,0
4,I can't breathe! was chosen as the most notabl...,1


In [ ]:
arq = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

Só rodar quando gerar o original

In [ ]:
repeticoes = 1

In [ ]:
# arq = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv'
# base_cru_1p.to_csv(arq, encoding='utf-8', index=False, sep=';', decimal = ',')

In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf

from nlpaug.util import Action

# Embedding Word2Vec


In [2]:
from nlpaug.util.file.download import DownloadUtil
DownloadUtil.download_glove('glove.6B', '.')

In [ ]:
aug_emb = naf.Sequential([naf.Sometimes([
    naw.WordEmbsAug(
    model_type='glove', model_path='./glove.6B.200d.txt',
    action="substitute")]),
    naf.Sometimes([naw.WordEmbsAug(
    model_type='glove', model_path='./glove.6B.200d.txt',
    action="insert"),
])])

In [ ]:
exp_emb = []

for i in range(repeticoes):
  base_cru_1p_emb = base_cru_1p.copy()
  base_cru_1p_emb['parafraseador'] = base_cru_1p_emb['texto'].progress_apply(lambda x:aug_emb.augment(x,num_gen))
  base_cru_1p_emb = base_cru_1p_emb.explode('parafraseador').reset_index(drop=True)
  exp_emb.append(base_cru_1p_emb)

  arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_emb'+str(i+1)+'.csv'
  exp_emb[i].to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

100%|██████████| 3817/3817 [2:31:20<00:00,  2.38s/it]
